In [1]:
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ManagedIdentityCredential

In [2]:
AZURE_DIRECTORY = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/aimethica-general/code/Users/alexandru.popescu/anomaly_DOCUMENTE_NIR_DETALII'

In [3]:
subscription_id = "9107822e-69ab-454f-a905-b59c4e15c1af"
resource_group = "AiMethica-ML"
workspace = "AiMethica-AZML"

In [4]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [5]:
ws = ml_client.workspaces.get(workspace)
print(ws.location, ":", ws.resource_group)

westeurope : AiMethica-ML


In [6]:
custom_env_name = 'aimethica-anomaly-env'
job_name = 'crimson_boat_5w5cz0r5kr'
env_name = custom_env_name + "@latest"

In [7]:
from azure.ai.ml.entities import Model

model = Model(
    # the script stores the model as "model"
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/outputs/",
    name="anomaly_document_facturi_export",
    description="Anomaly detection model for the DOCUMENT NIR DETALII dataset.",
    type="custom_model"
)

In [8]:
registered_model = ml_client.models.create_or_update(model=model)

In [9]:
online_endpoint_name = "anomaly-document-nir-detalii"

In [10]:
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Anomaly detection model for the DOCUMENT NIR DETALII dataset.",
    auth_mode="key"
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

print(f"Endpint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpint anomaly-document-nir-detalii provisioning state: Succeeded


In [11]:
model_name = registered_model.name + '@latest'

code_directory = os.path.join(AZURE_DIRECTORY,'src')

blue_deployment = ManagedOnlineDeployment(
    name="tff-blue",
    endpoint_name=online_endpoint_name,
    model=model_name,
    code_configuration=CodeConfiguration(code=code_directory, scoring_script="score.py"),
    environment=env_name,
    instance_type="Standard_DS1_v2",
    instance_count=1,
)

In [12]:
blue_deployment = ml_client.begin_create_or_update(blue_deployment)

Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint anomaly-document-nir-detalii exists
Uploading src (0.01 MBs): 100%|██████████| 9978/9978 [00:00<00:00, 314296.40it/s]




.

In [13]:
anomaly_request = os.path.join(AZURE_DIRECTORY,"data/anomaly_request.json")
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="tff-blue",
    request_file=anomaly_request
)

'{"response": [false, false, false, false]}'